In [17]:
import pandas as pd
import numpy as np
import wrds
import yfinance as yf
import datetime

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

In [18]:
db = wrds.Connection(wrds_username='zhan8289')

WRDS recommends setting up a .pgpass file.
You can find more info here:
https://www.postgresql.org/docs/9.5/static/libpq-pgpass.html.
There was an error with your password.


OperationalError: (psycopg2.OperationalError) fe_sendauth: no password supplied

(Background on this error at: http://sqlalche.me/e/14/e3q8)

In [8]:
# db.close()

In [9]:
# db.list_libraries()
# db.list_tables('crspa')
# db.describe_table('crsp', 'dsf')

In [10]:
house_members = pd.read_csv('house_members.csv')
house_committees = pd.read_excel('house_committees.xlsx', engine = 'openpyxl').rename(columns = {'House Member Name':'name', 'Commmitte':'committee', 'Committee #':'congress'})[['name', 'committee', 'congress']]
house_transactions = pd.read_csv('house_transactions.csv')
# change transaction date to date type
house_transactions['transaction_date'] = pd.to_datetime(house_transactions['transaction_date'])
# filter transactions on or after Jan 1 2020
house_transactions = house_transactions[house_transactions['transaction_date'] >= pd.to_datetime('2020-01-01')].sort_values(by = 'transaction_date').reset_index(drop = True)
# raw data has titles such as 'Hon. '
house_transactions.representative = house_transactions.representative.apply(lambda s: s[s.find(' ') + 1:])


senate_members = pd.read_excel('senate_members.xlsx', engine = 'openpyxl')
senate_committees = pd.read_excel('senate_committees.xlsx', engine = 'openpyxl', header = 1, usecols = 'B:D')
senate_transactions = pd.read_csv('senate_transactions.csv')
# change transaction date to date type
senate_transactions['transaction_date'] = pd.to_datetime(senate_transactions['transaction_date'])
# filter transactions on or after Jan 1 2020
senate_transactions = senate_transactions[senate_transactions['transaction_date'] >= pd.to_datetime('2020-01-01')].sort_values(by = 'transaction_date').reset_index(drop = True)
# only look at stocks
senate_transactions = senate_transactions[senate_transactions['asset_type'] == 'Stock'].reset_index(drop = True)

ImportError: Missing optional dependency 'openpyxl'.  Use pip or conda to install openpyxl.

In [4]:
tickers = pd.concat([house_transactions['ticker'], senate_transactions['ticker']]).drop_duplicates().sort_values()

In [5]:
prices = db.raw_sql('''
select date, htick, abs(prc) as prc from crsp.dsf a, (select permno, htick from crsp.dsfhdr
where htick in {}) b
where a.permno = b.permno
and date >= '2020-01-01'
'''.format(tuple(tickers)))

# db.raw_sql('''
# select permco, permno, htick, hcomnam from crsp.dsfhdr
# where htick in {}
# '''.format(tuple(stocks)))

In [6]:
def get_returns(ticker, date, time):
    times = {'1d':1, '5d':5, '1m':22}
    stock_prices = prices[(prices['date'] >= pd.to_datetime(date)) & (prices['htick'] == ticker)]
    try:
        return stock_prices.iloc[times[time], 2] / stock_prices.iloc[0, 2] - 1
    except:
        return None

get_returns('AAPL', pd.to_datetime('2020-01-01'), '1d')

-0.00972203551987938

In [12]:
start = datetime.datetime.now()
house_transactions['ret1d'] = house_transactions.apply(lambda row: get_returns(row.ticker, row.transaction_date, '1d'), axis = 1)
house_transactions['ret5d'] = house_transactions.apply(lambda row: get_returns(row.ticker, row.transaction_date, '5d'), axis = 1)
house_transactions['ret1m'] = house_transactions.apply(lambda row: get_returns(row.ticker, row.transaction_date, '1m'), axis = 1)
datetime.datetime.now() - start

KeyboardInterrupt: 

In [7]:
start = datetime.datetime.now()
senate_transactions['ret1d'] = senate_transactions.apply(lambda row: get_returns(row.ticker, row.transaction_date, '1d'), axis = 1)
senate_transactions['ret5d'] = senate_transactions.apply(lambda row: get_returns(row.ticker, row.transaction_date, '5d'), axis = 1)
senate_transactions['ret1m'] = senate_transactions.apply(lambda row: get_returns(row.ticker, row.transaction_date, '1m'), axis = 1)
datetime.datetime.now() - start

datetime.timedelta(seconds=185, microseconds=749846)

In [15]:
senate_transactions.to_csv('senate_transactions_w_returns.csv', index = False)

In [11]:
len(senate_transactions), len(house_transactions)

(1301, 9095)

In [ ]:
len(house_transactions), len(house_transactions[house_transactions['ret1m'].notna()]), len(senate_transactions), len(senate_transactions[senate_transactions['ret1m'].notna()])

In [7]:
# senate_transactions[senate_transactions['ticker'] != '--'][['ticker', 'asset_description']].sort_values(by = 'ticker').drop_duplicates()